In [1]:
import numpy as np
import pandas as pd 
import spacy
from spacy import displacy
import ast
import re
import nltk
from nltk.stem import WordNetLemmatizer
import sklearn
import calendar

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [5]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
movies = movies.merge(credits,on = 'title')

In [7]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [8]:
# columns to be removed

# Budget 
# homepage 
# overview  
# tagline 
# original_langauge  
# original_title 
# popularity
# Production company
# production countries
# release_date
# revenue
# spoken_language
# tagline
# runtime
# vote average
# vote count
# movie_id  

In [9]:
movies = movies[['id','title','overview','genres','keywords','cast','crew','spoken_languages','release_date']]

In [10]:
movies.columns

Index(['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew',
       'spoken_languages', 'release_date'],
      dtype='object')

In [11]:
movies.dropna(inplace=True)

In [12]:
movies.isnull().sum()

id                  0
title               0
overview            0
genres              0
keywords            0
cast                0
crew                0
spoken_languages    0
release_date        0
dtype: int64

In [13]:
# pd.to_datetime(movies.iloc[0].release_date)
movies['release_date'] = pd.to_datetime(movies['release_date'])

In [18]:
movies['release_date']

0      2009-12-10
1      2007-05-19
2      2015-10-26
3      2012-07-16
4      2012-03-07
          ...    
4804   1992-09-04
4805   2011-12-26
4806   2013-10-13
4807   2012-05-03
4808   2005-08-05
Name: release_date, Length: 4805, dtype: datetime64[ns]

In [19]:
movies['year'] = movies['release_date'].dt.year

In [20]:
movies['Month'] = movies['release_date'].dt.month
# movies['Month'] = calendar.month_name[2]

In [21]:
def month_to_name(obj):
    month_n  = calendar.month_name[obj]
    return month_n

In [23]:
movies['Month_Na'] = movies['Month'].apply(month_to_name)

In [24]:
# dw_mapping={
#     0: 'Monday', 
#     1: 'Tuesday', 
#     2: 'Wednesday', 
#     3: 'Thursday', 
#     4: 'Friday',
#     5: 'Saturday', 
#     6: 'Sunday'
# } 
# df['day_of_week_name']=df['DoB'].dt..map(dw_mapping)
# df
movies

,id,title,overview,genres,keywords,cast,crew,spoken_languages,release_date,year,Month,Month_Na
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",2009-12-10,2009,12,December
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",2007-05-19,2007,5,May
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",2015-10-26,2015,10,October
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",2012-07-16,2012,7,July
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",2012-03-07,2012,3,March
...,...,...,...,...,...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...","[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...","[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",1992-09-04,1992,9,September
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",[],2011-12-26,2011,12,December
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",2013-10-13,2013,10,October
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",2012-05-03,2012,5,May


In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
def Extract_tags(obj):
    L = []
    for i in eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(Extract_tags)

In [ ]:
movies.head(1)

In [ ]:
movies['keywords'] = movies['keywords'].apply(Extract_tags)

In [ ]:
movies.head(1)

In [ ]:
movies.iloc[0].cast

In [ ]:
def Extract_cast(obj):
    L = []
    count = 0
    for i in eval(obj):
        if count > 4 :
            break
        L.append(i['name'])
        count = count + 1
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(Extract_cast)

In [ ]:
movies.head(1)

In [ ]:
Id_overview = movies[['id','overview']]
pickle.dump(Id_overview,open("ID_overview.pkl","wb"))

In [ ]:
movies.iloc[0].crew

In [ ]:
def Extract_crew_director(obj):
    L = []
    for i in eval(obj):
        if i['job'] == "Director" :
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(Extract_crew_director)

In [ ]:
movies.head(1)

In [ ]:
# text = movies.iloc[1].overview
# nlp = spacy.load("en_core_web_sm")
# doc = nlp(text)
# for token in doc:
#     # check token pos
#     if token.pos_=='NOUN':
#         # print token
#         print(token.text)

In [ ]:
# for token in doc:
#     # extract subject
#     if (token.dep_=='nsubj'):
#         print(token.text)
#     # extract object
#     elif (token.dep_=='dobj'):
#         print(token.text)

In [ ]:
# text = movies.iloc[4].overview
# nlp = spacy.load("en_core_web_sm")
# doc = nlp(text)
# for token in doc:
#     # check token pos
#     if token.pos_=='NOUN':
#         # print token
#         print(token.text)

In [ ]:
# for token in doc:
#     # extract subject
#     if (token.dep_=='nsubj'):
#         print(token.text)
#     # extract object
#     elif (token.dep_=='dobj'):
#         print(token.text)

In [ ]:
nlp = spacy.load("en_core_web_lg")
nlp_sm = spacy.load("en_core_web_sm")

In [ ]:
# doc = nlp(movies.iloc[1].overview)

In [ ]:
# dir(doc)

In [ ]:

# displacy.render(doc, style="ent", jupyter=True)

In [ ]:
# [(i.text, i.ent_iob_ + "-" + i.ent_type_) for i in doc[0:30]]

In [ ]:
# all_orgs = []
# ans = []
# doc = nlp(movies.iloc[3].overview)
# for d in doc:
#     orgs = [d.text for ent in d.ent_type_ if d.ent_type_ == "PERSON" or d.ent_type_ == "ORG" or d.ent_type_ == "NORP" or d.ent_type_ == "EVENT" or d.ent_type_ == "WORK_OF_ART"]
#     all_orgs.extend(orgs)
# l = Counter(all_orgs).most_common(10)
# for i in l:
#     ans.append(i[0])

In [ ]:
from collections import Counter
def NRE_overview(obj):
    all_orgs = []
    ans = []
    doc = nlp(obj)
    for d in doc:
        orgs = [d.text for ent in d.ent_type_ if d.ent_type_ == "PERSON" or d.ent_type_ == "ORG" or d.ent_type_ == "NORP" or d.ent_type_ == "EVENT" or d.ent_type_ == "WORK_OF_ART"]
        all_orgs.extend(orgs)
    l = Counter(all_orgs).most_common(15)
    for i in l:
        ans.append(i[0])    
    return ans

In [ ]:
movies['overview'] = movies['overview'].apply(NRE_overview)

In [ ]:
# len(movies)

In [ ]:
# just_text = [movies.iloc[i].overview for i in range(4806)]
# docs = list(tqdm(nlp.pipe(just_text), total=len(just_text)))

In [ ]:
# from collections import Counter

# all_orgs = []
# for d in docs:
#     orgs = [ent.text for ent in d.ents if ent.label_ == "ORG"]
#     all_orgs.extend(orgs)

# l = Counter(all_orgs).most_common(15)
# l

In [ ]:
movies.head(1)

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x : [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x : [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x : [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x : [i.replace(" ","") for i in x])

In [ ]:
movies.head(1)

In [ ]:
movies['tags'] = movies['overview'] + movies['cast'] + movies['crew'] + movies['keywords'] + movies['genres']

In [ ]:
movies.head(1)

In [ ]:
Updated_dataframe = movies[['id','title','tags']]

In [ ]:
Updated_dataframe

In [ ]:
Updated_dataframe['tags'] = Updated_dataframe['tags'].apply(lambda x : " ".join(x))

In [ ]:
Updated_dataframe.head(7)

In [ ]:
Updated_dataframe['tags'] = Updated_dataframe['tags'].apply(lambda x : x.lower())

In [ ]:
Updated_dataframe.head(10)

In [ ]:
Updated_dataframe.iloc[7].tags

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
def remove_stop_words(obj):
    doc = obj
    doc = doc.lower()
    doc = doc.split()
    doc = [lemmatizer.lemmatize(word) for word in doc if not word in set(stopwords)]
    doc = ' '.join(doc)
    return doc

In [ ]:
# movies['tags'] = movies['tags'].apply(remove_stop_words)
Updated_dataframe['tags'] = Updated_dataframe['tags'].apply(remove_stop_words)

In [ ]:
movies.iloc[0].tags[0]

In [ ]:
Updated_dataframe.head()

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i)) 
    return " ".join(y)

In [ ]:
Updated_dataframe['tags'] = Updated_dataframe['tags'].apply(stem)

In [ ]:
Updated_dataframe['tags']

In [ ]:
list_tags = list(Updated_dataframe['tags'])

In [ ]:
list_tags

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer() 
vectors = vectorizer.fit_transform(list_tags)

In [ ]:
print("n_samples: %d, n_features: %d" % vectors.shape)

In [ ]:
# Select the first five documents from the data set
tf_idf = pd.DataFrame(vectors.todense())
tf_idf.columns = vectorizer.get_feature_names_out()
tfidf_matrix = tf_idf.T
# columns are the movies
# rows are the features/tags extracted 
tfidf_matrix.columns = [Updated_dataframe.iloc[i].title for i in range(0, 4806)]
tfidf_matrix['count'] = tfidf_matrix.sum(axis=1)
# All  features to be used 
# *************  PLEASE CHECK WITH DIFFERNT NUMBER OF ALLOWED FEATURE , TOp 100 or something like that ***************
tfidf_matrix = tfidf_matrix.sort_values(by ='count', ascending=False)[:] 
# Print the whole matrix
print(tfidf_matrix.drop(columns=['count']))

In [ ]:
# tfidf_matrix.to_csv('TF-IDF.csv',index = True)

In [ ]:
tfidf_matrix.columns

In [ ]:
tfidf_matrix.drop(['count'], axis=1,inplace=True)

In [ ]:
Vectorirsed_data = tfidf_matrix.transpose()

In [ ]:
Vectorirsed_data

In [ ]:
Vectorirsed_data.iloc[0].sort_values()

In [ ]:
numpy_vector_form_data = Vectorirsed_data.values

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
interaction_vector_movies = cosine_similarity(numpy_vector_form_data)

In [ ]:
interaction_vector_movies[0]

In [ ]:
list(enumerate(interaction_vector_movies[0]))

In [ ]:
def recommend(movie):
    index_movie = Updated_dataframe[Updated_dataframe['title'] == movie].index[0]
    movie_vector =sorted( list( enumerate(interaction_vector_movies[index_movie])) , reverse=True , key = lambda x:x[1])
    for i in range(1,11):
        print(Updated_dataframe.iloc[movie_vector[i][0]].id)
        print(Updated_dataframe.iloc[movie_vector[i][0]].title)


In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle
pickle.dump(Updated_dataframe , open('Movies_re.pkl','wb'))

In [ ]:
Updated_dataframe['title'].values

In [ ]:
pickle.dump(interaction_vector_movies ,open("interaction_vector_movies.pkl",'wb'))